<div style="width:100%; overflow:hidden; background-color:#F1F1E6; padding: 10px; border-style: outset; color:#17469e">
    <div style="width: 80%; float: left;">
    <h2 align="center">Universidad de Sonora</h2>
    <hr style="border-width: 3px; border-color:#17469e">
          <h1>Reconocimiento de patrones: Preparación de los datos</h1>          
          <h4>Ramón Soto C. <a href="mailto:rsotoc@moviquest.com/">(rsotoc@moviquest.com)</a></h4>
    </div>
    <div style="float: right;">
    <img src="images/escudo_unison.png">
    </div>
</div>

## Caso de estudio: [*Stack Overflow 2018 Developer Survey*](https://www.kaggle.com/stackoverflow/stack-overflow-2018-developer-survey)

Como caso de estudio principal en el presente curso hemos seleccionado la encuesta de desarrolladores 2018 de *Stack Overflow* disponible en [Kaggle](https://www.kaggle.com). En este primer análisis, realizaremos las fases de comprensión del negocio y comprensión de los datos.

### 3. Preparación de los datos

<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7; ">

Los datos de nuestro caso de estudio, después de la limpieza hecha en [2.2. Preparación de los datos II](2.4.%20Preparación%20de%20los%20datos%20IV.ipynb), son los siguientes:

</div>

In [1]:
"""
Reconocimiento de patrones: Preparación de los datos
"""

import pandas as pd
import numpy as np
import json
import pickle

from collections import Counter
from operator import itemgetter
from IPython.display import display, HTML

pd.set_option('display.max_columns', 130)
pd.set_option('max_colwidth', 80)

In [2]:
path = "Data sets/Stack Overflow Survey/"

# Recuperar encabezados de columnas en orden original
with open(path + 'survey_results_public_clean.headers', 'rb') as file:  
    headers = pickle.load(file)

with open(path + 'survey_results_public_clean.json') as f:
    dict_json = json.load(f)
df = pd.DataFrame.from_dict(dict_json)

# Reordenar las columnas de acuerdo al orden original
df = df.reindex(headers, axis=1)

print(df.info(), "\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98443 entries, 0 to 98442
Columns: 120 entries, Hobby to SurveyEasy
dtypes: float64(41), object(79)
memory usage: 90.1+ MB
None 



### <span style="color:#313f9e; font-weight: bold;">Transformación de los datos:</span>
<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7;">
La transformación de los datos está orientada a reescribir los datos en forma más adecuada para su procesamiento.
<br>
Uno de los principales problemas en nuestros datos es que 81 columnas aparecen clasificadas como tipo <code style="background-color:#f7f7f7;">object</code>, es decir, texto. 

<br>Entre estos casos, el primer problema que atacaremos será el de los reactivos con opción múltiple.

<br>
Consideremos, por ejemplo el caso de la variable 'LanguageWorkedWith': 
</div>

In [3]:
def get_values(col_name):
    full_list = ";".join(col_name)
    each_word = full_list.split(";")
    each_word = sorted(list(Counter(each_word).keys()))
    return each_word

In [4]:
print(df['LanguageWorkedWith'].describe())

values = get_values(df['LanguageWorkedWith'].dropna())
print(values)

count                          78334
unique                         26678
top       C#;JavaScript;SQL;HTML;CSS
freq                            1347
Name: LanguageWorkedWith, dtype: object
['Assembly', 'Bash/Shell', 'C', 'C#', 'C++', 'CSS', 'Clojure', 'Cobol', 'CoffeeScript', 'Delphi/Object Pascal', 'Erlang', 'F#', 'Go', 'Groovy', 'HTML', 'Hack', 'Haskell', 'Java', 'JavaScript', 'Julia', 'Kotlin', 'Lua', 'Matlab', 'Objective-C', 'Ocaml', 'PHP', 'Perl', 'Python', 'R', 'Ruby', 'Rust', 'SQL', 'Scala', 'Swift', 'TypeScript', 'VB.NET', 'VBA', 'Visual Basic 6']


<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7;">
Obsérvese que las respuestas, que en los datos originales se representan por cadenas, deberían estar representadas por listas de valores o por vectores: 
</div>

In [5]:
# Obtener lista ordenada de valores
values = get_values(df['LanguageWorkedWith'].dropna())

# Crear diccionario
values_dict = {}
# Crear diccionario inverso
indices_dict = {}    
for i in range(len(values)):
    values_dict[str(i)] = values[i]
    indices_dict[values[i]] = str(i)

# Convertir cadena de términos separados por ; en lista ordenada
def get_vectors(col_name):
    words_list = col_name.split(";")
    words_vector = []
    for w in words_list:
        words_vector.append(indices_dict[w])
    return sorted(words_vector)

# Generar nueva columna
df['LWW_vect'] = df.apply(
    lambda row: 
    [] if pd.isnull(row.LanguageWorkedWith) else get_vectors(row.LanguageWorkedWith), 
    axis=1
)

# Guardar el diccionario
dict_of_dicts = {'LanguageWorkedWith': values_dict}

display(df.head()[['LanguageWorkedWith', 'LWW_vect']], dict_of_dicts)

,LanguageWorkedWith,LWW_vect
0,JavaScript;Python;HTML;CSS,"[14, 18, 27, 5]"
1,JavaScript;Python;Bash/Shell,"[1, 18, 27]"
2,None,[]
3,C#;JavaScript;SQL;TypeScript;HTML;CSS;Bash/Shell,"[1, 14, 18, 3, 31, 34, 5]"
4,C;C++;Java;Matlab;R;SQL;Bash/Shell,"[1, 17, 2, 22, 28, 31, 4]"


{'LanguageWorkedWith': {'0': 'Assembly',
  '1': 'Bash/Shell',
  '10': 'Erlang',
  '11': 'F#',
  '12': 'Go',
  '13': 'Groovy',
  '14': 'HTML',
  '15': 'Hack',
  '16': 'Haskell',
  '17': 'Java',
  '18': 'JavaScript',
  '19': 'Julia',
  '2': 'C',
  '20': 'Kotlin',
  '21': 'Lua',
  '22': 'Matlab',
  '23': 'Objective-C',
  '24': 'Ocaml',
  '25': 'PHP',
  '26': 'Perl',
  '27': 'Python',
  '28': 'R',
  '29': 'Ruby',
  '3': 'C#',
  '30': 'Rust',
  '31': 'SQL',
  '32': 'Scala',
  '33': 'Swift',
  '34': 'TypeScript',
  '35': 'VB.NET',
  '36': 'VBA',
  '37': 'Visual Basic 6',
  '4': 'C++',
  '5': 'CSS',
  '6': 'Clojure',
  '7': 'Cobol',
  '8': 'CoffeeScript',
  '9': 'Delphi/Object Pascal'}}

<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7;">
La vieja columna 'LanguageWorkedWith' ahora es innecesaria, la eliminamos y renombramos 'LWW_vect' como 'LanguageWorkedWith':
</div>

In [6]:
df = df.drop(['LanguageWorkedWith'], axis=1)
df = df.rename({'LWW_vect':'LanguageWorkedWith'}, axis='columns')

display(df.head()['LanguageWorkedWith'])

0              [14, 18, 27, 5]
1                  [1, 18, 27]
2                           []
3    [1, 14, 18, 3, 31, 34, 5]
4    [1, 17, 2, 22, 28, 31, 4]
Name: LanguageWorkedWith, dtype: object

<div style="border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7;">
Un inconveniente de esta transformación, es que las herramientas básicas de exploración, como <code style="background-color:#f7f7f7;">pandas.DataFrame.describe()</code> no se pueden emplear:
</div>

<br>

<div style="width:10%; float:left; padding-right:5px; color:#3442a0; font-family: Courier">
In [xx]:
</div>
<div>
<div style="margin-left: 10%; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7; font-family: Courier">
<span style="background-color:#f7f7f7; color:#008002">print</span>(df.describe(include='all'))
</div>

<div style="margin-left: 10%; padding: 8px 12px; font-family: Courier;">

---------------------------------------------------------------------------
<code style="color: #8b0101;">
TypeError                                 </code>Traceback (most recent call last)
<br>
...
<br><span style="color: #8b0101;">TypeError: </span><span style="color: #4582b4;">unhashable type: 'list'</span>

</div></div>

<div style="border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7;">
Podemos hacer la misma operación con las variables 'DevType', 'CommunicationTools', 'EducationTypes', 'SelfTaughtTypes', 'HackathonReasons', 'LanguageDesireNextYear', 'DatabaseWorkedWith', 'DatabaseDesireNextYear', 'PlatformWorkedWith', 'PlatformDesireNextYear', 'FrameworkWorkedWith', 'FrameworkDesireNextYear', 'IDE', 'Methodology', 'VersionControl', 'AdBlockerReasons', 'AdsActions', 'ErgonomicDevices' y  'RaceEthnicity'. Antes re-escribimos las funciones a utilizar en la transformación:
</div>

In [7]:
def get_vectors(col_name, dictionary):
    words_list = col_name.split(";")
    words_vector = []
    for w in words_list:
        words_vector.append(dictionary[w])
    return sorted(words_vector)

def vectorize(col_name):
    values = get_values(df[col_name].dropna())
        
    values_dict = {}
    indices_dict = {}    
    for i in range(len(values)):
        values_dict[str(i)] = values[i]
        indices_dict[values[i]] = str(i)
    dict_of_dicts[col_name] = values_dict
    
    df[col_name] = df.apply(
        lambda row: 
        [] if pd.isnull(row[col_name]) else get_vectors(row[col_name], indices_dict), 
        axis=1
    )

    return

columns = ['DevType', 'CommunicationTools', 'EducationTypes', 'SelfTaughtTypes', 
           'HackathonReasons', 'LanguageDesireNextYear', 'DatabaseWorkedWith', 
           'DatabaseDesireNextYear', 'PlatformWorkedWith', 'PlatformDesireNextYear', 
           'FrameworkWorkedWith', 'FrameworkDesireNextYear', 'IDE', 'Methodology', 
           'VersionControl', 'AdBlockerReasons', 'AdsActions', 'ErgonomicDevices',
           'RaceEthnicity']

for col in columns:
    vectorize(col)

<div style="border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7;">
Otro tipo de representación que resultaría conveniente modificar, es la de variables como 'AssessJob1'-'AssessJob10'. Estas 10 variables en realidad corresponden a una sola pregunta, en la que los encuestados ordenan de acuerdo a su importancia, diferentes aspectos de una oferta de empleo. En estos casos, sería conveniente reemplazar el conjunto de variables por una sola variable vectorial con las respuestas de las variables originales. En el caso ejemplo, reemplazar el conjunto de variables 'AssessJob1'-'AssessJob10' por una sola variable 'AssessJob' que describa la importancia relativa de cada uno de los 10 aspectos contemplados
(convertimos de flotante a caracter):
</div>

In [8]:
df['AssessJob'] = df.apply(
    lambda row: 
    [] if pd.isnull(row.AssessJob1) 
    else [str(int(row.AssessJob1)), str(int(row.AssessJob2)), str(int(row.AssessJob3)), 
          str(int(row.AssessJob4)), str(int(row.AssessJob5)), str(int(row.AssessJob6)), 
          str(int(row.AssessJob7)), str(int(row.AssessJob8)), str(int(row.AssessJob9)), 
          str(int(row.AssessJob10))], 
    axis=1
)

display(df.head()['AssessJob'])
df = df.drop(['AssessJob1', 'AssessJob2', 'AssessJob3', 'AssessJob4', 'AssessJob5', 
              'AssessJob6', 'AssessJob7', 'AssessJob8', 'AssessJob9', 'AssessJob10'], axis=1)

0    [10, 7, 8, 1, 2, 5, 3, 4, 9, 6]
1    [1, 7, 10, 8, 2, 5, 4, 3, 6, 9]
2                                 []
3                                 []
4    [8, 5, 7, 1, 2, 6, 4, 3, 10, 9]
Name: AssessJob, dtype: object

<div style="border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7;">
Nuevamente, repetimos la operación para otras variables con la msima estructura: 'AssessBenefits1'-'	AssessBenefits11', 'JobContactPriorities1'-'JobContactPriorities5', 'JobEmailPriorities1'-'JobEmailPriorities7' y 'AdsPriorities1'-'AdsPriorities7'; las variables 'AdsAgreeDisagree?', 'HypotheticalTools?' y 'AgreeDisagree?' tienen diferente cantidad de valores faltantes, lo cual refleja que tienen otro tipo de naturaleza.
</div>

In [9]:
def column_vector(row, columns):
    vector = []
    for col in columns:
        vector.append(str(int(row[col])))
    return vector

names = ['AssessBenefits', 'JobContactPriorities', 'JobEmailPriorities', 'AdsPriorities']
ranges = [12, 6, 8, 8]

for n in range(4):
    columns = []
    for i in range(1, ranges[n]):
        columns.append(names[n] + str(i))
    
    df[names[n]] = df.apply(
        lambda row: 
        [] if pd.isnull(row[columns[0]]) 
        else column_vector(row, columns), 
        axis=1
    )
    
    df = df.drop(list(columns), axis=1)
    display(df.head()[names[n]])

0                                     []
1    [1, 5, 3, 7, 10, 4, 11, 9, 6, 2, 8]
2                                     []
3                                     []
4    [1, 10, 2, 4, 8, 3, 11, 7, 5, 9, 6]
Name: AssessBenefits, dtype: object

0    [3, 1, 4, 2, 5]
1    [3, 1, 5, 2, 4]
2                 []
3                 []
4    [2, 1, 4, 5, 3]
Name: JobContactPriorities, dtype: object

0    [5, 6, 7, 2, 1, 4, 3]
1    [1, 3, 4, 5, 2, 6, 7]
2                       []
3                       []
4    [7, 3, 6, 2, 1, 4, 5]
Name: JobEmailPriorities, dtype: object

0    [1, 5, 4, 7, 2, 6, 3]
1    [3, 5, 1, 4, 6, 7, 2]
2                       []
3                       []
4    [2, 3, 4, 6, 1, 7, 5]
Name: AdsPriorities, dtype: object

<div style="border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7;">
Las variables 'StackOverflowRecommend' y 'StackOverflowJobsRecommend' presentan una característica que dificulta su análisis y que son innecesarias: En los valores inicial y final se especifica el significado del valor, '0 (Not Likely)' y '10 (Very Likely)'. Simplifiquemos la notación: 
</div>

In [10]:
df['StackOverflowRecommend'] = df.apply(
    lambda row: 
    np.nan if pd.isnull(row.StackOverflowRecommend) 
    else row.StackOverflowRecommend.split(' ')[0], 
    axis=1
)

df['StackOverflowJobsRecommend'] = df.apply(
    lambda row: 
    np.nan if pd.isnull(row.StackOverflowJobsRecommend) 
    else row.StackOverflowJobsRecommend.split(' ')[0], 
    axis=1
)

<div style="border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7;">
En la mayoría de las variables se utilizan variables lingüísticas. Esta representación tiene dos problemas, el primero es que tal representación es muy susceptible a pequeños errores y a "desviaciones de escritura". 
<br>Consideremos, por ejemplo el caso de la variable 'Country'; esta variable incluye valores como '<i>Democratic People's Republic of Korea</i>' y '<i>North Korea</i>' que corresponden al mismo país. Para mejorar la representación emplearemos el código <a href='https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3'>ISO 3166-1 alpha-3</a>. Este código utiliza 3 letras para representar cada país (hemos agregado el código 'NLC' para la etiqueta 'Other Country (Not Listed Above)'):
</div>

In [11]:
countries_df = pd.read_csv(path + "countries_codes.csv", 
                 names = ['Country', 'Code'])
display(HTML(countries_df.to_html()))

,Country,Code
0,Afghanistan,AFG
1,Åland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM
5,Andorra,AND
6,Angola,AGO
7,Anguilla,AIA
8,Antarctica,ATA
9,Antigua and Barbuda,ATG


In [12]:
# Crear diccionario a partir del Dataframe
countries_dict = dict(zip(countries_df['Country'].tolist(), countries_df['Code'].tolist()))

print('Antes:\n', df['Country'].describe())
df['Country'] = df.apply(
    lambda row: "" if pd.isnull(row.Country) else countries_dict[row.Country], axis=1
)
print('\nDespués:\n', df['Country'].describe())

dict_of_dicts['Country'] = values_dict

Antes:
 count             98443
unique              183
top       United States
freq              20309
Name: Country, dtype: object

Después:
 count     98443
unique      181
top         USA
freq      20309
Name: Country, dtype: object


<div style="border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7;">
El segundo problema del uso de una representación basada en etiquetas lingüísticas es que  para determinar su semejanza se utilizan funciones como la distancia de Hamming o la similaridad de Gower, que realizan la comparación caracter por caracter, de manera que la longitud de cadenas influye en el desempeño del algoritmo. Consideremos, por ejemplo, el siguiente posible valor de la variable 'StackOverflowJobs':
</div>

In [13]:
print(df.loc[0, 'StackOverflowJobs'], '\nLongitud =', len(df.loc[0, 'StackOverflowJobs']))

No, I knew that Stack Overflow had a jobs board but have never used or visited it 
Longitud = 81


<div style="border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7;">
Re-codificaremos las columnas restantes tipo texto, utilizando tablas de índices:
</div>

In [14]:
columns = [
    'Hobby', 'OpenSource', 'Student', 'Employment', 'FormalEducation', 'UndergradMajor',
    'CompanySize', 'YearsCoding', 'YearsCodingProf', 'HopeFiveYears', 'JobSearchStatus', 
    'LastNewJob', 'UpdateCV', 'TimeFullyProductive', 'OperatingSystem', 'AdBlocker', 
    'CheckInCode', 'AdBlockerDisable', 'AIDangerous', 'AgreeDisagree1', 'AgreeDisagree2',
    'AgreeDisagree3', 'AIInteresting', 'AIResponsible', 'AIFuture', 'EthicsChoice', 
    'EthicsReport', 'EthicsResponsible', 'EthicalImplications', 'StackOverflowVisit', 
    'WakeTime', 'StackOverflowHasAccount', 'StackOverflowParticipate', 'Exercise', 
    'StackOverflowJobs', 'StackOverflowDevStory', 'HoursComputer', 'HypotheticalTools1', 
    'HypotheticalTools2', 'HypotheticalTools3', 'HypotheticalTools4', 'Dependents', 
    'HypotheticalTools5', 'StackOverflowConsiderMember', 'HoursOutside', 'SkipMeals', 
    'EducationParents', 'SurveyTooLong', 'SurveyEasy', 'JobSatisfaction', 'Age', 
    'CareerSatisfaction', 'AdsAgreeDisagree1', 'AdsAgreeDisagree2', 'AdsAgreeDisagree3'
]

for col in columns:
    values = get_values(df[col].dropna())
    values_dict = {}
    indices_dict = {}
    for i in range(len(values)):
        values_dict[str(i)] = values[i]
        indices_dict[values[i]] = str(i)
    
    df[col] = df.apply(
        lambda row: 
        np.nan if pd.isnull(row[col]) else indices_dict[row[col]], 
        axis=1
    )
    dict_of_dicts[col] = values_dict

print(df.head())

  Hobby OpenSource Country Student Employment FormalEducation UndergradMajor  \
0     1          0     KEN       0          1               1             10   
1     1          1     GBR       0          0               1              3   
2     1          1     USA       0          0               0              6   
3     0          0     USA       0          0               1              6   
4     1          0     ZAF       3          0               8              6   

  CompanySize             DevType YearsCoding YearsCodingProf JobSatisfaction  \
0           4                [12]           7               7               1   
1           2      [12, 20, 4, 7]           8               3               2   
2           4            [10, 12]           5               9               3   
3           3                [12]           3               1               4   
4           2  [13, 18, 19, 2, 6]           9               0               7   

  CareerSatisfaction HopeFiveYea

<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7;">
La única variable numérica que mantenemos es 'ConvertedSalary':
</div>

In [15]:
print(df.ConvertedSalary.describe())

count     88045.000000
mean      20174.654045
std       31345.350033
min           0.000000
25%           0.000000
50%           0.000000
75%       36716.000000
max      199950.000000
Name: ConvertedSalary, dtype: float64


<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7;">
En este caso, por la magnitud de los valores, es conveniente hacer una normalización. Durante la fase de limpieza, un tope máximo de $200,000.00USD anuales (y el mínimo debe ser 0), podemos entonces, manteniendo este valor como máximo permitido, escalar los datos entre 0 y 1:
</div>

In [16]:
df.ConvertedSalary = df.ConvertedSalary / 200000

print(df.ConvertedSalary.describe())

count    88045.000000
mean         0.100873
std          0.156727
min          0.000000
25%          0.000000
50%          0.000000
75%          0.183580
max          0.999750
Name: ConvertedSalary, dtype: float64


<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7;">
Por lo pronto no hacemos más transformaciones. Guardamos los datos modificados:
</div>

In [17]:
# Guardar los encabezados del dataframe en el orden actual
with open(path + "survey_results_public_transformed.headers", 'wb') as file:  
    pickle.dump(list(df), file)

# Guardar los diccionarios
with open(path + "survey_results_public_transformed.dicts", 'wb') as file:  
    pickle.dump(dict_of_dicts, file)

# Guardar en formato JSON (como diccionario)
df.to_json(path + "survey_results_public_transformed.json", orient='records') 

###### <a name="tarea_8">Tarea 8</a>

Presente el reporte de transformación de los datos para su caso de estudio.

**Fecha de entrega**: 8 de octubre.